<a href="https://colab.research.google.com/github/mohammadsjahanbakhsh/project2_quera/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## preprocessing

In [1]:
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:


def transform(dataset,country_column) :
       
        
        cols_with_na = dataset.isna().sum()[dataset.isna().sum()>0].index.tolist()
        
        for col in cols_with_na : 
          
          dataset[col]=dataset.groupby([country_column])[col].transform(lambda x: x.fillna(x.mean()))
          
        cols_with_na = dataset.isna().sum()[dataset.isna().sum()>0].index.tolist()
        
        for col in cols_with_na : 
            dataset.loc[:,col].fillna(dataset[col].median(),inplace=True)
            
        
        return dataset



def transform2(dataset,drop_list):
  dataset=dataset[dataset.Year>=1950]
  dataset.drop(drop_list,axis=1,inplace=True)

  return dataset

Additional_column=["Variant","Notes","ISO3 Alpha-code","ISO2 Alpha-code","SDMX code**","Type","Location code","Parent code"]

In [3]:
death=pd.read_csv("datasets\suicide-vs-violent-deaths.csv")

In [4]:
death=transform2(death,['Population (historical estimates)', 'Continent',"Code"])
# death=transform(death,'Entity')

C:\Users\mj749\AppData\Local\Temp\ipykernel_50940\3742734326.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.drop(drop_list,axis=1,inplace=True)


In [5]:
vaccine=pd.read_csv("datasets\child-mortality-vs-share-of-children-immunized-against-diphtheria-pertussis-and-tetanus.csv")
vaccine=transform2(vaccine,['Population (historical estimates)', 'Continent','Code',"Mortality rate, under-5 (per 1,000 live births)"])
# vaccine=transform(vaccine,'Entity')

C:\Users\mj749\AppData\Local\Temp\ipykernel_50940\3742734326.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.drop(drop_list,axis=1,inplace=True)


In [6]:
gdp=pd.read_csv("datasets\life-expectancy-vs-gdp-per-capita.csv")
gdp.columns

Index(['Entity', 'Code', 'Year', 'Life expectancy at birth (historical)',
       'GDP per capita', '417485-annotations',
       'Population (historical estimates)', 'Continent'],
      dtype='object')

In [7]:
gdp=transform2(gdp,['Code', 'Life expectancy at birth (historical)','417485-annotations','Population (historical estimates)','Continent'])
# gdp=transform(gdp,'Entity')

C:\Users\mj749\AppData\Local\Temp\ipykernel_50940\3742734326.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.drop(drop_list,axis=1,inplace=True)


In [8]:
Estimates=pd.read_excel("datasets/WPP2022.xlsx",sheet_name="Estimates")


In [9]:
from numpy import nan

columns=list(Estimates.iloc[15])
Estimates=Estimates[16:]
Estimates.columns=columns
Estimates=Estimates.set_index("Index")

# Estimates=Estimates.loc[Estimates.iloc[:,4].notna(),:]
Estimates.replace("...",nan,inplace=True)

In [10]:
x=vaccine.merge(death,how="outer").merge(gdp,how="outer")

Estimates=Estimates.merge(x,left_on=["Region, subregion, country or area *","Year"],right_on=["Entity","Year"],how="left").drop("Entity",axis=1)
x

,Entity,Year,DTP3 (% of one-year-olds immunized),Deaths - Self-harm - Sex: Both - Age: Age-standardized (Rate),Deaths - All forms of violence - OWID - Sex: Both - Age: Age-standardized (Rate),GDP per capita
0,Abkhazia,2015,NaN,NaN,NaN,NaN
1,Afghanistan,1960,NaN,NaN,NaN,1326.000
2,Afghanistan,1961,NaN,NaN,NaN,1309.000
3,Afghanistan,1962,NaN,NaN,NaN,1302.000
4,Afghanistan,1963,NaN,NaN,NaN,1298.000
...,...,...,...,...,...,...
20254,Yugoslavia,2013,NaN,NaN,NaN,14500.000
20255,Yugoslavia,2014,NaN,NaN,NaN,14627.000
20256,Yugoslavia,2016,NaN,NaN,NaN,15416.000
20257,Yugoslavia,2017,NaN,NaN,NaN,15960.843


In [11]:
hospital_bed=pd.read_csv("datasets/hospital_bed.csv")
hospital_bed.drop(["INDICATOR","SUBJECT","MEASURE","FREQUENCY","Flag Codes"],axis=1,inplace=True)
hospital_bed.rename({"TIME":"Year","Value":"hospital_bed"},axis=1,inplace=True)

In [12]:
Estimates=Estimates.merge(hospital_bed,left_on=["ISO3 Alpha-code","Year"],right_on=["LOCATION","Year"],how="left").drop("LOCATION",axis=1)

In [13]:

Estimates.drop(Additional_column,axis=1,inplace=True)



In [14]:
Medium_variant=pd.read_excel("datasets/WPP2022.xlsx",sheet_name="Medium variant")

In [15]:
columns=list(Medium_variant.iloc[15])
Medium_variant=Medium_variant[16:]
Medium_variant.columns=columns
Medium_variant=Medium_variant.set_index("Index")
Medium_variant=Medium_variant[Medium_variant.iloc[:,4].notna()]
# Medium_variant.drop(Additional_column,axis=1,inplace=True)
Medium_variant=Medium_variant[Medium_variant.Year==2022]
Medium_variant.replace("...",nan,inplace=True)

In [16]:
Estimates.to_csv("Estimates3.csv",index=None)
# Medium_variant.to_csv("/content/drive/MyDrive/data/Medium_variant.csv",index=None)
# فایلو بدم بچه ها اشتباه داشت

In [17]:
Estimates

,"Region, subregion, country or area *",Year,"Total Population, as of 1 January (thousands)","Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)","Median Age, as of 1 July (years)","Natural Change, Births minus Deaths (thousands)",...,"Mortality between Age 15 and 60, both sexes (deaths under age 60 per 1,000 alive at age 15)","Male Mortality between Age 15 and 60 (deaths under age 60 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 60 (deaths under age 60 per 1,000 females alive at age 15)",Net Number of Migrants (thousands),"Net Migration Rate (per 1,000 population)",DTP3 (% of one-year-olds immunized),Deaths - Self-harm - Sex: Both - Age: Age-standardized (Rate),Deaths - All forms of violence - OWID - Sex: Both - Age: Age-standardized (Rate),GDP per capita,hospital_bed
0,WORLD,1950.0,2477674.732,2499322.157,1245362.965,1253959.192,19.166,99.314,22.224,43294.722,...,378.697,430.259,324.931,0.000,0.000,NaN,NaN,NaN,NaN,NaN
1,WORLD,1951.0,2520969.582,2543130.380,1267473.182,1275657.198,19.502,99.358,22.172,44322.348,...,368.319,415.836,319.336,0.000,0.000,NaN,NaN,NaN,NaN,NaN
2,WORLD,1952.0,2565291.179,2590270.899,1291399.982,1298870.917,19.864,99.425,22.108,49960.072,...,353.055,395.533,309.910,0.000,0.000,NaN,NaN,NaN,NaN,NaN
3,WORLD,1953.0,2615250.618,2640278.797,1316659.222,1323619.574,20.247,99.474,22.036,50056.756,...,345.083,385.843,303.905,0.000,0.000,NaN,NaN,NaN,NaN,NaN
4,WORLD,1954.0,2665306.975,2691979.339,1342923.171,1349056.168,20.643,99.545,21.952,53345.249,...,335.442,374.658,295.994,0.000,0.000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22020,Wallis and Futuna Islands,2017.0,12.002,11.936,5.769,6.167,85.257,93.562,33.847,0.072,...,82.852,82.197,83.320,-0.201,-16.706,NaN,NaN,NaN,NaN,NaN
22021,Wallis and Futuna Islands,2018.0,11.870,11.816,5.703,6.112,84.396,93.301,34.397,0.066,...,79.633,88.556,72.163,-0.171,-14.368,NaN,NaN,NaN,NaN,NaN
22022,Wallis and Futuna Islands,2019.0,11.761,11.714,5.648,6.066,83.671,93.109,34.962,0.072,...,66.639,69.324,64.496,-0.171,-14.489,NaN,NaN,NaN,NaN,NaN
22023,Wallis and Futuna Islands,2020.0,11.667,11.655,5.614,6.040,83.246,92.955,35.453,0.060,...,72.730,76.411,69.685,-0.083,-7.096,NaN,NaN,NaN,NaN,NaN


In [18]:
Estimates=pd.read_csv("Estimates.csv")
Medium_variant=pd.read_csv("Medium_variant.csv")

## model 

In [19]:
import pandas as pd
Estimates=pd.read_csv("Estimates.csv")

In [20]:


def transform(dataset,country_column) :
       
        
        cols_with_na = dataset.isna().sum()[dataset.isna().sum()>0].index.tolist()
        
        for col in cols_with_na : 
          
          dataset[col]=dataset.groupby([country_column])[col].transform(lambda x: x.fillna(x.mean()))
          
        cols_with_na = dataset.isna().sum()[dataset.isna().sum()>0].index.tolist()
        
        for col in cols_with_na : 
            dataset.loc[:,col].fillna(dataset[col].median(),inplace=True)
            
        
        return dataset


Additional_column=["Variant","Notes","ISO3 Alpha-code","ISO2 Alpha-code","SDMX code**","Type","Location code","Parent code"]

In [21]:
Medium_variant=pd.read_csv("Medium_variant.csv")

In [22]:

Estimates.drop(Additional_column,axis=1,inplace=True)

In [23]:
df_est=pd.get_dummies(transform(Estimates,"Region, subregion, country or area *"),columns=["Region, subregion, country or area *"])
# df_est=pd.get_dummies(Estimates,columns=["Region, subregion, country or area *"])

In [24]:
Medium_variant=Medium_variant.drop(Additional_column,axis=1)

In [25]:
df_med=pd.get_dummies(transform(Medium_variant,"Region, subregion, country or area *"),columns=["Region, subregion, country or area *"])
# df_med=pd.get_dummies(Medium_variant,columns=["Region, subregion, country or area *"])


In [26]:
# x_col_est=["Life Expectancy" not in c for c in df_est.columns]
# y_col_est=["Life Expectancy at Birth" in c for c in df_est.columns]
# x=df_est.loc[:,x_col_est].drop(my_feature,axis=1)
# y=df_est.loc[:,y_col_est]["Life Expectancy at Birth, both sexes (years)"]


In [27]:
x_col_est=["Life Expectancy" not in c for c in df_est.columns]
y_col_est=["Life Expectancy at Birth" in c for c in df_est.columns]
df_est_x=df_est.loc[:,x_col_est]
df_est_y=df_est.loc[:,y_col_est]["Life Expectancy at Birth, both sexes (years)"]


In [28]:
x_col_med=["Life Expectancy" not in c for c in df_med.columns]
y_col_med=["Life Expectancy at Birth" in c for c in df_med.columns]
df_med_x=df_med.loc[:,x_col_med]
df_med_y=df_med.loc[:,y_col_med]["Life Expectancy at Birth, both sexes (years)"]


## predict my feature with decision tree Regressor 



In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import r2_score

In [31]:
df_est.head(2)

,Year,"Total Population, as of 1 January (thousands)","Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)","Median Age, as of 1 July (years)","Natural Change, Births minus Deaths (thousands)","Rate of Natural Change (per 1,000 population)",...,"Region, subregion, country or area *_Uruguay","Region, subregion, country or area *_Uzbekistan","Region, subregion, country or area *_Vanuatu","Region, subregion, country or area *_Venezuela (Bolivarian Republic of)","Region, subregion, country or area *_Viet Nam","Region, subregion, country or area *_Wallis and Futuna Islands","Region, subregion, country or area *_Western Sahara","Region, subregion, country or area *_Yemen","Region, subregion, country or area *_Zambia","Region, subregion, country or area *_Zimbabwe"
0,1950,2229.322,2254.113,1079.773,1174.340,86.864,91.947,18.315,62.914,27.788,...,0,0,0,0,0,0,0,0,0,0
1,1951,2278.903,2303.248,1104.543,1198.705,88.757,92.145,18.084,61.906,26.799,...,0,0,0,0,0,0,0,0,0,0


In [32]:
df_est_x.head(2)

,Year,"Total Population, as of 1 January (thousands)","Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)","Median Age, as of 1 July (years)","Natural Change, Births minus Deaths (thousands)","Rate of Natural Change (per 1,000 population)",...,"Region, subregion, country or area *_Uruguay","Region, subregion, country or area *_Uzbekistan","Region, subregion, country or area *_Vanuatu","Region, subregion, country or area *_Venezuela (Bolivarian Republic of)","Region, subregion, country or area *_Viet Nam","Region, subregion, country or area *_Wallis and Futuna Islands","Region, subregion, country or area *_Western Sahara","Region, subregion, country or area *_Yemen","Region, subregion, country or area *_Zambia","Region, subregion, country or area *_Zimbabwe"
0,1950,2229.322,2254.113,1079.773,1174.340,86.864,91.947,18.315,62.914,27.788,...,0,0,0,0,0,0,0,0,0,0
1,1951,2278.903,2303.248,1104.543,1198.705,88.757,92.145,18.084,61.906,26.799,...,0,0,0,0,0,0,0,0,0,0


In [33]:
my_feature_total=Estimates.columns[-5:].to_list()

In [34]:
my_feature=my_feature_total.copy()
my_feature2=[]

In [35]:
r2_my_feature=dict()
medium_my_feature = dict()

In [36]:
my_feature

['DTP3 (% of one-year-olds immunized)',
 'Deaths - Self-harm - Sex: Both - Age: Age-standardized (Rate)',
 'Deaths - All forms of violence - OWID - Sex: Both - Age: Age-standardized (Rate)',
 'GDP per capita',
 'hospital_bed']

In [37]:
# x_last=df_est.drop(my_feature_total,axis=1)
# y_last=df_est[[my_feature2[-1]]]
# my_feature2.append(  "2 " + my_feature2[-1] )

In [39]:
for i in range(len(my_feature_total)):
  x_last=df_est_x.drop(my_feature_total,axis=1)
  my_feature2.append(my_feature.pop(-1))
  y_last=df_est_x[[my_feature2[-1]]]

  x_train,x_test , y_train,y_test = train_test_split(x_last, y_last ,test_size=.2, random_state=33)

  std_scaler = StandardScaler().fit(x_train)
  x_train = std_scaler.transform(x_train)


  
  regressor = DecisionTreeRegressor(random_state = 33)
  regressor.fit(x_train, y_train)
  print(my_feature2[-1])

  y_pred = regressor.predict(std_scaler.transform(x_test))


  r2_my_feature[my_feature2[-1]]=r2_score(y_test, y_pred)


  if r2_my_feature[my_feature2[-1]] > 0.8:
    medium_my_feature[my_feature2[-1]] = regressor.predict(std_scaler.transform(df_med_x))


else:
  for i,j in r2_my_feature.items():
    print(i," : ", j)

hospital_bed
GDP per capita
Deaths - All forms of violence - OWID - Sex: Both - Age: Age-standardized (Rate)
Deaths - Self-harm - Sex: Both - Age: Age-standardized (Rate)
DTP3 (% of one-year-olds immunized)
hospital_bed  :  -0.12554231134938276
GDP per capita  :  0.9712392847139224
Deaths - All forms of violence - OWID - Sex: Both - Age: Age-standardized (Rate)  :  0.4855004660779235
Deaths - Self-harm - Sex: Both - Age: Age-standardized (Rate)  :  0.9828520815789328
DTP3 (% of one-year-olds immunized)  :  0.8287366523387778


In [40]:
my_feature_in_medium=list(medium_my_feature.keys())
my_feature_in_medium

['GDP per capita',
 'Deaths - Self-harm - Sex: Both - Age: Age-standardized (Rate)',
 'DTP3 (% of one-year-olds immunized)']

In [41]:
pd.DataFrame(medium_my_feature)

,GDP per capita,Deaths - Self-harm - Sex: Both - Age: Age-standardized (Rate),DTP3 (% of one-year-olds immunized)
0,819.894799,16.392333,93.000000
1,1082.071043,6.830000,85.000000
2,3171.928558,9.905000,59.000000
3,7025.930000,15.280333,94.000000
4,925.494399,15.422000,65.000000
...,...,...,...
232,7025.930000,14.930000,68.000000
233,7025.930000,10.160000,88.857143
234,7025.930000,5.180000,94.000000
235,7025.930000,18.095667,99.000000


In [42]:
df_med_x

,Year,"Total Population, as of 1 January (thousands)","Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)","Median Age, as of 1 July (years)","Natural Change, Births minus Deaths (thousands)","Rate of Natural Change (per 1,000 population)",...,"Region, subregion, country or area *_Uruguay","Region, subregion, country or area *_Uzbekistan","Region, subregion, country or area *_Vanuatu","Region, subregion, country or area *_Venezuela (Bolivarian Republic of)","Region, subregion, country or area *_Viet Nam","Region, subregion, country or area *_Wallis and Futuna Islands","Region, subregion, country or area *_Western Sahara","Region, subregion, country or area *_Yemen","Region, subregion, country or area *_Zambia","Region, subregion, country or area *_Zimbabwe"
0,2022,12715.870,12889.576,6400.926,6488.650,496.708,98.648,15.766,345.407,26.798,...,0,0,0,0,0,0,0,0,0,0
1,2022,829.245,836.774,420.357,416.416,449.636,100.946,20.261,17.056,20.359,...,0,0,0,0,0,0,0,0,0,0
2,2022,1113.147,1120.849,556.264,564.584,48.354,98.526,23.980,14.508,12.949,...,0,0,0,0,0,0,0,0,0,0
3,2022,3652.541,3684.032,1817.878,1866.154,30.410,97.413,18.413,80.139,21.699,...,0,0,0,0,0,0,0,0,0,0
4,2022,121822.590,123379.924,61980.175,61399.749,123.380,100.945,18.647,3126.668,25.338,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,2022,220.728,222.382,113.375,109.007,78.580,104.007,20.639,4.808,21.547,...,0,0,0,0,0,0,0,0,0,0
233,2022,1.861,1.871,0.935,0.937,155.917,99.786,26.662,0.021,11.246,...,0,0,0,0,0,0,0,0,0,0
234,2022,106.399,106.858,52.848,54.010,164.396,97.849,21.714,1.717,16.009,...,0,0,0,0,0,0,0,0,0,0
235,2022,11.267,11.312,5.799,5.513,377.050,105.197,25.032,0.150,13.215,...,0,0,0,0,0,0,0,0,0,0


In [43]:

df_med_x=pd.concat([df_med_x,pd.DataFrame(medium_my_feature)],axis=1)
df_med_x

,Year,"Total Population, as of 1 January (thousands)","Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)","Median Age, as of 1 July (years)","Natural Change, Births minus Deaths (thousands)","Rate of Natural Change (per 1,000 population)",...,"Region, subregion, country or area *_Venezuela (Bolivarian Republic of)","Region, subregion, country or area *_Viet Nam","Region, subregion, country or area *_Wallis and Futuna Islands","Region, subregion, country or area *_Western Sahara","Region, subregion, country or area *_Yemen","Region, subregion, country or area *_Zambia","Region, subregion, country or area *_Zimbabwe",GDP per capita,Deaths - Self-harm - Sex: Both - Age: Age-standardized (Rate),DTP3 (% of one-year-olds immunized)
0,2022,12715.870,12889.576,6400.926,6488.650,496.708,98.648,15.766,345.407,26.798,...,0,0,0,0,0,0,0,819.894799,16.392333,93.000000
1,2022,829.245,836.774,420.357,416.416,449.636,100.946,20.261,17.056,20.359,...,0,0,0,0,0,0,0,1082.071043,6.830000,85.000000
2,2022,1113.147,1120.849,556.264,564.584,48.354,98.526,23.980,14.508,12.949,...,0,0,0,0,0,0,0,3171.928558,9.905000,59.000000
3,2022,3652.541,3684.032,1817.878,1866.154,30.410,97.413,18.413,80.139,21.699,...,0,0,0,0,0,0,0,7025.930000,15.280333,94.000000
4,2022,121822.590,123379.924,61980.175,61399.749,123.380,100.945,18.647,3126.668,25.338,...,0,0,0,0,0,0,0,925.494399,15.422000,65.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,2022,220.728,222.382,113.375,109.007,78.580,104.007,20.639,4.808,21.547,...,0,0,0,0,0,0,0,7025.930000,14.930000,68.000000
233,2022,1.861,1.871,0.935,0.937,155.917,99.786,26.662,0.021,11.246,...,0,0,0,0,0,0,0,7025.930000,10.160000,88.857143
234,2022,106.399,106.858,52.848,54.010,164.396,97.849,21.714,1.717,16.009,...,0,0,0,0,0,0,0,7025.930000,5.180000,94.000000
235,2022,11.267,11.312,5.799,5.513,377.050,105.197,25.032,0.150,13.215,...,0,0,0,0,0,0,0,7025.930000,18.095667,99.000000


In [44]:
df_med_x

,Year,"Total Population, as of 1 January (thousands)","Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)","Median Age, as of 1 July (years)","Natural Change, Births minus Deaths (thousands)","Rate of Natural Change (per 1,000 population)",...,"Region, subregion, country or area *_Venezuela (Bolivarian Republic of)","Region, subregion, country or area *_Viet Nam","Region, subregion, country or area *_Wallis and Futuna Islands","Region, subregion, country or area *_Western Sahara","Region, subregion, country or area *_Yemen","Region, subregion, country or area *_Zambia","Region, subregion, country or area *_Zimbabwe",GDP per capita,Deaths - Self-harm - Sex: Both - Age: Age-standardized (Rate),DTP3 (% of one-year-olds immunized)
0,2022,12715.870,12889.576,6400.926,6488.650,496.708,98.648,15.766,345.407,26.798,...,0,0,0,0,0,0,0,819.894799,16.392333,93.000000
1,2022,829.245,836.774,420.357,416.416,449.636,100.946,20.261,17.056,20.359,...,0,0,0,0,0,0,0,1082.071043,6.830000,85.000000
2,2022,1113.147,1120.849,556.264,564.584,48.354,98.526,23.980,14.508,12.949,...,0,0,0,0,0,0,0,3171.928558,9.905000,59.000000
3,2022,3652.541,3684.032,1817.878,1866.154,30.410,97.413,18.413,80.139,21.699,...,0,0,0,0,0,0,0,7025.930000,15.280333,94.000000
4,2022,121822.590,123379.924,61980.175,61399.749,123.380,100.945,18.647,3126.668,25.338,...,0,0,0,0,0,0,0,925.494399,15.422000,65.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,2022,220.728,222.382,113.375,109.007,78.580,104.007,20.639,4.808,21.547,...,0,0,0,0,0,0,0,7025.930000,14.930000,68.000000
233,2022,1.861,1.871,0.935,0.937,155.917,99.786,26.662,0.021,11.246,...,0,0,0,0,0,0,0,7025.930000,10.160000,88.857143
234,2022,106.399,106.858,52.848,54.010,164.396,97.849,21.714,1.717,16.009,...,0,0,0,0,0,0,0,7025.930000,5.180000,94.000000
235,2022,11.267,11.312,5.799,5.513,377.050,105.197,25.032,0.150,13.215,...,0,0,0,0,0,0,0,7025.930000,18.095667,99.000000


In [45]:
df_med_y



0      61.977
1      63.680
2      62.859
3      66.604
4      65.645
        ...  
232    72.598
233    75.380
234    71.270
235    64.854
236    80.015
Name: Life Expectancy at Birth, both sexes (years), Length: 237, dtype: float64

In [46]:
pd.concat([df_med_x,df_med_y],axis=1).to_csv("Medium_variant_with_my_feature.csv",index=None)


# predicting Life Expectancy at Birth, both sexes 

## decesion tree in sklearn

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [48]:
df=pd.read_csv("Medium_variant_with_my_feature.csv")

In [49]:
df_med_x=df.iloc[:,:-1]
df_med_y=df.iloc[:,-1]
df_med_x

,Year,"Total Population, as of 1 January (thousands)","Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)","Median Age, as of 1 July (years)","Natural Change, Births minus Deaths (thousands)","Rate of Natural Change (per 1,000 population)",...,"Region, subregion, country or area *_Venezuela (Bolivarian Republic of)","Region, subregion, country or area *_Viet Nam","Region, subregion, country or area *_Wallis and Futuna Islands","Region, subregion, country or area *_Western Sahara","Region, subregion, country or area *_Yemen","Region, subregion, country or area *_Zambia","Region, subregion, country or area *_Zimbabwe",GDP per capita,Deaths - Self-harm - Sex: Both - Age: Age-standardized (Rate),DTP3 (% of one-year-olds immunized)
0,2022,12715.870,12889.576,6400.926,6488.650,496.708,98.648,15.766,345.407,26.798,...,0,0,0,0,0,0,0,819.894799,16.392333,93.000000
1,2022,829.245,836.774,420.357,416.416,449.636,100.946,20.261,17.056,20.359,...,0,0,0,0,0,0,0,1082.071043,6.830000,85.000000
2,2022,1113.147,1120.849,556.264,564.584,48.354,98.526,23.980,14.508,12.949,...,0,0,0,0,0,0,0,3171.928558,9.905000,59.000000
3,2022,3652.541,3684.032,1817.878,1866.154,30.410,97.413,18.413,80.139,21.699,...,0,0,0,0,0,0,0,7025.930000,15.280333,94.000000
4,2022,121822.590,123379.924,61980.175,61399.749,123.380,100.945,18.647,3126.668,25.338,...,0,0,0,0,0,0,0,925.494399,15.422000,65.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,2022,220.728,222.382,113.375,109.007,78.580,104.007,20.639,4.808,21.547,...,0,0,0,0,0,0,0,7025.930000,14.930000,68.000000
233,2022,1.861,1.871,0.935,0.937,155.917,99.786,26.662,0.021,11.246,...,0,0,0,0,0,0,0,7025.930000,10.160000,88.857143
234,2022,106.399,106.858,52.848,54.010,164.396,97.849,21.714,1.717,16.009,...,0,0,0,0,0,0,0,7025.930000,5.180000,94.000000
235,2022,11.267,11.312,5.799,5.513,377.050,105.197,25.032,0.150,13.215,...,0,0,0,0,0,0,0,7025.930000,18.095667,99.000000


In [50]:
df_med_y

0      61.977
1      63.680
2      62.859
3      66.604
4      65.645
        ...  
232    72.598
233    75.380
234    71.270
235    64.854
236    80.015
Name: Life Expectancy at Birth, both sexes (years), Length: 237, dtype: float64

In [51]:
x_train,x_test , y_train,y_test = train_test_split(df_med_x, df_med_y ,test_size=.2, random_state=33)

std_scaler = StandardScaler().fit(x_train)
x_train = std_scaler.transform(x_train)


  


In [52]:
regressor = DecisionTreeRegressor(random_state = 33)
regressor.fit(x_train, y_train)


y_pred = regressor.predict(std_scaler.transform(x_test))


r2_score(y_test, y_pred)

0.9520965404041678

## RandomForestRegressor in sklearn

In [53]:
x_train,x_test , y_train,y_test = train_test_split(df_med_x, df_med_y ,test_size=.2, random_state=33)

std_scaler = StandardScaler().fit(x_train)
x_train = std_scaler.transform(x_train)


  


In [54]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000, random_state = 33)

rf.fit(x_train, y_train);

predictions = rf.predict(std_scaler.transform(x_test))
r2_score(y_test, predictions)

0.9795615889259432

## LinearRegression

In [55]:
from sklearn.linear_model import LinearRegression

In [56]:
reg = LinearRegression()
reg.fit(x_train,y_train)

LinearRegression()

In [57]:
reg_pred = reg.predict(std_scaler.transform(x_test))

In [58]:
r2_score(y_test, reg_pred)


0.9458479457854545

## AutoML

In [59]:
! pip install flaml

In [60]:
x_train,x_test , y_train,y_test = train_test_split(df_med_x, df_med_y ,test_size=.2, random_state=33)

std_scaler = StandardScaler().fit(x_train)
x_train = std_scaler.transform(x_train)


In [61]:
from flaml import AutoML

automl = AutoML(task='regression', time_budget=60)
automl.fit(x_train, y_train)

[flaml.automl.automl: 02-25 18:50:38] {2716} INFO - task = regression
[flaml.automl.automl: 02-25 18:50:38] {2718} INFO - Data split method: uniform
[flaml.automl.automl: 02-25 18:50:38] {2721} INFO - Evaluation method: cv
[flaml.automl.automl: 02-25 18:50:38] {2848} INFO - Minimizing error metric: 1-r2
[flaml.automl.automl: 02-25 18:50:38] {2994} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.automl: 02-25 18:50:38] {3323} INFO - iteration 0, current learner lgbm
[flaml.automl.automl: 02-25 18:50:38] {3461} INFO - Estimated sufficient time budget=2573s. Estimated necessary time budget=18s.
[flaml.automl.automl: 02-25 18:50:38] {3508} INFO -  at 0.3s,	estimator lgbm's best error=0.4989,	best estimator lgbm's best error=0.4989
[flaml.automl.automl: 02-25 18:50:38] {3323} INFO - iteration 1, current learner lgbm
[flaml.automl.automl: 02-25 18:50:38] {3508} INFO -  at 0.5s,	estimator lgbm's best error=0.4989,	best estimato

In [62]:
y_pred = automl.predict(std_scaler.transform(x_test))
r2_score(y_test, y_pred)

0.9792524382146618

In [63]:
# time 20 -->  0.9801760763529554



## xgboost

In [64]:

import xgboost as xgb


In [65]:
x_train,x_test , y_train,y_test = train_test_split(df_med_x, df_med_y ,test_size=.2, random_state=33)

std_scaler = StandardScaler().fit(x_train)
x_train = std_scaler.transform(x_train)


In [66]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 4, alpha = 10, n_estimators = 1000)

In [67]:
xg_reg.fit(x_train,y_train)

xgb_pred = xg_reg.predict(std_scaler.transform(x_test))

[18:51:38] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [68]:
xgb_score = r2_score(y_test, xgb_pred)
xgb_score

0.97540795383377